<a href="https://colab.research.google.com/github/Jyothics/Deep-Learning-Basic-to-Advance-/blob/master/MP_Neuron_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [2]:
# read data from file
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [0]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [0]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [0]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [6]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [0]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [0]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [9]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [0]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [11]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [12]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'SIM Slot(s)', 'Resolution', 'Sim1', 'Capacity',
       'Internal Memory', 'Pixel Density', 'RAM', 'Num_cores', 'os_name',
       'Weight', 'Processor_frequency', 'Brand', 'SIM 2', 'Height',
       'Screen to Body Ratio (calculated)', 'Screen Size'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Resolution', 'Capacity', 'Internal Memory', 'Pixel Density',
       'RAM', 'Weight', 'Processor_frequency', 'Height',
       'Screen to Body Ratio (calculated)', 'Screen Size',
       'SIM Slot(s)_Dual SIM, GSM+CDMA', 'SIM Slot(s)_Dual SIM, GSM+GSM',
       'SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE',
       'SIM Slot(s)_Single SIM, GSM', 'Sim1_2G', 'Sim1_3G', 'Sim1_4G',
       'Num_cores_312', 'Num_cores_Deca', 'Num_cores_Dual', 'Num_cores_Hexa',
       'Num_cores_Octa', 'Num_cores_Other', 'Num_cores_Quad',
       'Num_cores_Tru-Octa', 'os_name_Android', 'os_name_Blackberry',
       'os_name_KAI', 'os_name_Nokia', 'os_name_Other', 'os_name_Tizen',
       'os_na

In [0]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [14]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [15]:
train_new.head()

,PhoneId,Resolution,Capacity,Internal Memory,Pixel Density,RAM,Weight,Processor_frequency,Height,Screen to Body Ratio (calculated),...,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other,Rating
0,0,20,4000,64,403,4,182,1.8,157.9,80.68,...,0,1,0,0,0,0,0,1,0,4.5
1,1,8,4230,32,271,3,168,1.8,156.2,80.85,...,0,0,0,0,0,0,0,1,0,4.5
2,2,25,3500,32,409,3,168,2.1,157.0,83.68,...,0,0,0,0,0,0,0,1,0,4.4
3,4,24,3300,64,411,4,169,2.2,159.8,74.78,...,0,0,0,0,0,0,0,1,0,4.3
4,5,16,3750,64,396,4,175,2.2,160.4,84.23,...,0,0,0,0,0,0,0,1,0,4.4


In [16]:
test_new.head()

,PhoneId,Resolution,Capacity,Internal Memory,Pixel Density,RAM,Weight,Processor_frequency,Height,Screen to Body Ratio (calculated),...,Brand_VOTO,Brand_Vivo,Brand_Xiaomi,Brand_Xiaomi Poco,Brand_Yu,Brand_iVooMi,SIM 2_2G,SIM 2_3G,SIM 2_4G,SIM 2_Other
0,3,5,4230,16,271,2,168,1.8,156.2,80.85,...,0,0,0,0,0,0,0,0,1,0
1,11,12,5000,64,402,4,205,1.8,156.0,81.60,...,0,0,0,0,0,0,0,0,1,0
2,13,25,3500,64,409,6,169,2.0,156.7,83.84,...,0,0,0,0,0,0,0,0,1,0
3,16,16,3500,64,409,4,169,2.0,156.7,83.84,...,0,0,0,0,0,0,0,0,1,0
4,19,20,4000,64,403,4,181,1.8,158.6,77.43,...,0,0,1,0,0,0,0,0,1,0


## Dummy Solution

In [0]:
X = train_new.drop(['Rating','PhoneId','SIM 2_3G','Sim1_3G','os_name_Blackberry','os_name_Tizen','os_name_KAI','Num_cores_Deca','SIM Slot(s)_Dual SIM, GSM+CDMA','Brand_Lyf','Brand_Karbonn','Brand_Jivi','Brand_Lephone','Brand_LG','Brand_Micromax','Brand_Reliance', 'Brand_Mobiistar', 'Brand_Nubia', 'Brand_Razer', 'Brand_Spice', 'Brand_Ulefone', 'Brand_Yu', 'Brand_VOTO', 'Brand_Do', 'Brand_InFocus', 'Brand_Intex', 'Brand_HTC', 'Brand_Coolpad', 'Brand_Billion', 'Brand_iVooMi', 'Brand_10.or','Brand_Blackberry'], axis=1)

In [0]:
X_test = test_new.drop(['PhoneId','SIM 2_3G','Sim1_3G','os_name_Blackberry','os_name_Tizen','os_name_KAI','Num_cores_Deca','SIM Slot(s)_Dual SIM, GSM+CDMA', 'Brand_Lyf','Brand_Karbonn','Brand_Jivi','Brand_Lephone','Brand_LG','Brand_Micromax','Brand_Reliance', 'Brand_Mobiistar', 'Brand_Nubia', 'Brand_Razer', 'Brand_Spice', 'Brand_Ulefone', 'Brand_Yu', 'Brand_VOTO', 'Brand_Do', 'Brand_InFocus', 'Brand_Intex', 'Brand_HTC', 'Brand_Coolpad', 'Brand_Billion', 'Brand_iVooMi', 'Brand_10.or','Brand_Blackberry'], axis=1)

In [19]:
X.shape

(341, 56)

In [0]:
X['RAM'] = X['RAM'].map(lambda x: 0 if(x<2 or x>511) else 1)
X['Pixel Density'] = X['Pixel Density'].map(lambda x: 0 if x<234 else 1)
X['Weight'] = X['Weight'].map(lambda x: 0 if x>160 else 1)
X['Height'] = X['Height'].map(lambda x: 0 if (x>156.5 and x<=157.9) else 1)
X['Resolution'] = X['Resolution'].map(lambda x: 1 if x>8 else 0)
X['Screen to Body Ratio (calculated)'] = X['Screen to Body Ratio (calculated)'].map(lambda x: 1 if x>79.8 else 0)
X['Screen Size'] = X['Screen Size'].map(lambda x: 0 if x<5.6 else 1)
X['Capacity'] = X['Capacity'].map(lambda x:1 )
X['Internal Memory'] = X['Internal Memory'].map(lambda x:1 if x<32 else 0)
X['Processor_frequency'] = X['Processor_frequency'].map(lambda x:0 if x<1.6 else 1)

In [0]:
X_test['RAM'] = X_test['RAM'].map(lambda x: 0 if(x<2 or x>511) else 1)
X_test['Pixel Density'] = X_test['Pixel Density'].map(lambda x: 0 if x<234 else 1)
X_test['Weight'] = X_test['Weight'].map(lambda x: 0 if x>165 else 1)
X_test['Height'] = X_test['Height'].map(lambda x: 0 if (x>156.5 and x<=157.9) else 1)
X_test['Resolution'] = X_test['Resolution'].map(lambda x: 1 if x<8 else 0)
X_test['Screen to Body Ratio (calculated)'] = X_test['Screen to Body Ratio (calculated)'].map(lambda x: 1 if x>80 else 0)
X_test['Screen Size'] = X_test['Screen Size'].map(lambda x: 0 if x<=5.5 else 1)
X_test['Capacity'] = X_test['Capacity'].map(lambda x:1 )    
X_test['Internal Memory'] = X_test['Internal Memory'].map(lambda x:1 if x<32 else 0)
X_test['Processor_frequency'] = X_test['Processor_frequency'].map(lambda x:0 if x<1.6 else 1)

In [0]:
Y = train_new['Rating']
Y = Y.map(lambda x: 0 if x<4.0 else 1)

In [0]:
X_binarised_train = X.apply(pd.cut, bins=2, labels=[0,1])
X_binarised_test = X_test.apply(pd.cut, bins=2, labels=[0,1])

In [0]:
X_binarised_train = X_binarised_train.values
X_binarised_test = X_binarised_test.values

# X_binarised_train.head()

In [0]:
class MPNeuron:
  
  def __init__(self):
    self.b = None

  def model(self, x):
    if (sum(x) >= self.b):
       return 1
    else :
       return 0
  
  def predict(self, X):
    Y = []
    for x in X:
      result = self.model(x)
      Y.append(result)
    return np.array(Y)
  
  def fit(self, X, Y):
    accuracy = {}
    
    for b in range(X.shape[1] + 1):
      self.b = b
      Y_pred = self.predict(X)
      accuracy[b] = accuracy_score(Y_pred, Y)
      
    best_b = max(accuracy, key = accuracy.get)
    self.b = best_b
    
    print('Optimal value of b is', best_b)
    print('Highest accuracy is', accuracy[best_b])
    print(accuracy)

In [27]:
mp_neuron = MPNeuron()
mp_neuron.fit(X_binarised_train, Y)

Optimal value of b is 10
Highest accuracy is 0.8123167155425219
{0: 0.6979472140762464, 1: 0.6979472140762464, 2: 0.6979472140762464, 3: 0.6979472140762464, 4: 0.6979472140762464, 5: 0.6979472140762464, 6: 0.6979472140762464, 7: 0.7008797653958945, 8: 0.7096774193548387, 9: 0.7683284457478006, 10: 0.8123167155425219, 11: 0.7331378299120235, 12: 0.5659824046920822, 13: 0.40762463343108507, 14: 0.3225806451612903, 15: 0.3020527859237537, 16: 0.3020527859237537, 17: 0.3020527859237537, 18: 0.3020527859237537, 19: 0.3020527859237537, 20: 0.3020527859237537, 21: 0.3020527859237537, 22: 0.3020527859237537, 23: 0.3020527859237537, 24: 0.3020527859237537, 25: 0.3020527859237537, 26: 0.3020527859237537, 27: 0.3020527859237537, 28: 0.3020527859237537, 29: 0.3020527859237537, 30: 0.3020527859237537, 31: 0.3020527859237537, 32: 0.3020527859237537, 33: 0.3020527859237537, 34: 0.3020527859237537, 35: 0.3020527859237537, 36: 0.3020527859237537, 37: 0.3020527859237537, 38: 0.3020527859237537, 39: 0.30

In [0]:
Y_test_pred = mp_neuron.predict(X_binarised_test)

In [29]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':Y_test_pred})
submission = submission[['PhoneId', 'Class']]
submission.head()

,PhoneId,Class
0,3,1
1,11,1
2,13,1
3,16,1
4,19,1


In [0]:
submission.to_csv("submission1.csv", index=False)